In [2]:
import pickle
import vertexai
from vertexai.generative_models import (
    GenerationConfig,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)
from google.cloud import storage

In [3]:
project_id = "jesusarguelles-sandbox"
location = "us-central1"

In [4]:
vertexai.init(project=project_id, location=location)
get_product_sku = "get_product_sku"
bucket = storage.Client(project="jesusarguelles-sandbox").bucket("jesusarguelles-datasets-public")
blob = bucket.blob("money_laundering_detection/model.pkl")

with blob.open("rb") as f:
    api_model = pickle.load(f)
f.close()

blob = bucket.blob("money_laundering_detection/label_encoder.pkl")

with blob.open("rb") as f:
    label_mappings = pickle.load(f)

In [5]:
# Functions

get_laundering_prediction = FunctionDeclaration(
    name="get_laundering_prediction",
    description="Get if the money is laundered",
    # Function parameters are specified in OpenAPI JSON schema format
    parameters={
        "type": "object",
        "properties": {
            "segment": {"type": "integer", "description": "The segment number"},
            "step": {"type": "integer", "description": "The number of step"},
            "trans_type": {"type": "object", "description": "Transaction Type"},
            "amount": {"type": "number", "description": "The amount of the transaction", "format": "float"},
            "nameOrig": {"type": "object", "description": "The id, or name of the transfer origin"},
            "oldbalanceOrg": {"type": "number", "description": "The old balance organization number, oldbalancerOrg", "format": "float"},
            "nameDest": {"type": "object", "description": "Name or id of the transfer destination"},
            "oldbalanceDest": {"type": "number", "description": "old balance destination", "format": "float"},
            "accountType": {"type": "object", "description": "The account type"}
        },
    },
)

retail_tool = Tool(
    function_declarations=[
        get_laundering_prediction,
    ],
)

In [10]:
import pandas as pd
import vertexai.preview.generative_models as generative_models

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
}


textsi_1 = """Given the following Context respond as follows:
- Available tools: get_laundering_prediction

    segment: 0,
    step: 2,
    trans_type: WIRE_OUT,
    amount: 18627.02,
    nameOrig: C1375503918,
    oldbalanceOrg: 18627.02,
    nameDest: C234430897,
    oldbalanceDest: 0.0,
    accountType: FOREIGN
1 as laundered, 0 as not."""

model = GenerativeModel(
    model_name="gemini-1.0-pro-002",
    system_instruction=[textsi_1],
    generation_config=GenerationConfig(temperature=0),
    safety_settings=safety_settings,
    tools=[retail_tool],
)

# Start a chat session
chat = model.start_chat()

# Send a prompt for the first conversation turn that should invoke the get_product_sku function

def get_laundering_prediction(
        segment,
        step,
        trans_type,
        amount,
        nameOrig,
        oldbalanceOrg,
        nameDest,
        oldbalanceDest,
        accountType
):
    predict = {
        "segment": [segment],
        "step": [step],
        "trans_type": [trans_type],
        "amount": [amount],
        "nameOrig": [nameOrig],
        "oldbalanceOrg": [oldbalanceOrg],
        "nameDest": [nameDest],
        "oldbalanceDest": [oldbalanceDest],
        "accountType": [accountType]
    }
    to_predict_df = pd.DataFrame(predict)
    cat_feat = [i for i in to_predict_df.columns if to_predict_df[i].dtypes == 'O']
    print(cat_feat)

    for i in cat_feat:
        to_predict_df[i] = to_predict_df[i].map(label_mappings[i])
    print(to_predict_df)
    re = str(api_model.predict(to_predict_df)[0])
    return {"is_laundered": re}
# Check the function name that the model responded with, and make an API call to an external system

prompt = """
can you tell me if the model was laundered?
"""
response = chat.send_message(prompt)

try:
    function_call = response.candidates[0].function_calls[0]
    print(function_call.name)
    if function_call.name == "get_laundering_prediction":

        segment = function_call.args["segment"]
        step = function_call.args["step"]
        trans_type = function_call.args["trans_type"]
        amount = function_call.args["amount"]
        nameOrig = function_call.args["nameOrig"]
        oldbalanceOrg = function_call.args["oldbalanceOrg"]
        nameDest = function_call.args["nameDest"]
        oldbalanceDest = function_call.args["oldbalanceDest"]
        accountType = function_call.args["accountType"]
        api_response = get_laundering_prediction(
            segment,
            step,
            trans_type,
            amount,
            nameOrig,
            oldbalanceOrg,
            nameDest,
            oldbalanceDest,
            accountType
        )

    re = chat.send_message(
        Part.from_function_response(
            name="get_laundering_prediction",
            response={
                "content": api_response,
            },
        ),
    )
    re = re.text
    print(re)
except:
    try:
        re = response.candidates[0].content.parts[0].text
    except:
        re = "no response"
# # Extract the text from the model response
print(re)

get_laundering_prediction
['trans_type', 'nameOrig', 'nameDest', 'accountType']
   segment  step  trans_type    amount  nameOrig  oldbalanceOrg  nameDest  \
0      0.0   2.0           5  18627.02     40991       18627.02      2718   

   oldbalanceDest  accountType  
0             0.0            1  
The model predicts that the money was laundered.
The model predicts that the money was laundered.


In [7]:
"""
    segment: 0,
    step: 2,
    trans_type: WIRE_OUT,
    amount: 18627.02,
    nameOrig: C1375503918,
    oldbalanceOrg: 18627.02,
    nameDest: C234430897,
    oldbalanceDest: 0.0,
    accountType: FOREIGN
"""

d = {
    "segement": [0],
    "step": [2],
    "trans_type": ["WIRE_OUT"],
    "amount": [18627.02],
    "nameOrig": ["C1375503918"],
    "oldbalanceOrg": [18627.02],
    "nameDest": ["C234430897"],
    "oldbalanceDest": [0.0],
    "accountType": ["FOREIGN"]
}
to_predict_df = pd.DataFrame(d)
cat_feat = [i for i in to_predict_df.columns if to_predict_df[i].dtypes == 'O']
print(cat_feat)

['trans_type', 'nameOrig', 'nameDest', 'accountType']


In [24]:
label_mappings.keys()

dict_keys(['trans_type', 'nameOrig', 'nameDest', 'accountType'])

In [23]:
label_mappings["oldbalanceOrg"]

KeyError: 'oldbalanceOrg'